# II. Base Containers

In [1]:
import sys

sys.path.append("../../forme-groups-python-3-12/")

## 1. Create a Base Container

In [2]:
from app.src.base.container import BaseContainer

container_hello = BaseContainer({"hello": "world"}, "dictionary")
print(f'Representation: {repr(container_hello)}')
print(f'String: {container_hello}')

BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
Representation: BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=dict)
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
String: {'hello': 'world'}


Note: Base Containers are immutable. You cannot change the base container once it is created.

In [3]:
from attrs.exceptions import FrozenInstanceError

# Check that the value is frozen
try:
    container_hello._items = 'world2'
except FrozenInstanceError as e:
    print(f'Error: {e.msg}')

Error: can't set attribute


## 2. Base Container Types

Base Containers support the following types:
- **Dictionary** - A dictionary of key-value pairs.
- **List** - A list of values.
- **Tuple** - A tuple of values.  An immutable list.  Item order is preserved.
- **Set** - A set of values.  Sets contain unique values.  No values are duplicated.
- **Frozen Set** - A frozen set of values.  Frozen sets contain unique values.  No values are duplicated.


In [4]:
container_dict = BaseContainer({"hello": "world"}, "dictionary")
print(f'{repr(container_dict)}')

container_list = BaseContainer(["hello", "world"], "list")
print(f'{repr(container_list)}')

container_tuple = BaseContainer(("hello", "world"), "tuple")
print(f'{repr(container_tuple)}')

container_set = BaseContainer({"hello", "world"}, "set")
print(f'{repr(container_set)}')

container_frozenset = BaseContainer(frozenset({"hello", "world"}), "frozenset")
print(f'{repr(container_frozenset)}')

BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=dict)
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=list)
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=tuple)
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseContainer(items=(BaseValue(value='world', type=str), BaseValue(value='hello', type=str)), type=set)
BaseValueType: <attribute '__bound__' of 'ty

## 2A. Forcing Base Container Types

You can force a base container to be a specific type when it is created.

Items are stored in a container by deffault as Tuple of Base Values.  

In [5]:
container_dict = BaseContainer(("hello", "world"), "dictionary")
print(f'{repr(container_dict)}')
print(f'Container Type: {container_dict.type}')
print(f'Container Value: {container_dict}')


BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=dict)
Container Type: dict
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
BaseValueType: <attribute '__bound__' of 'typing.TypeVar' objects>
Container Value: {'hello': 'world'}
